In [22]:
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
import pandas as pd
import string
import numpy

# Creando el libro de excel
wb = Workbook()
ws = wb.create_sheet('Reporte', index = 0)

#Propiedades
ws.sheet_view.showGridLines = False
ws.sheet_view.zoomScale = 70

for column in ["A","B","C","D","E","F","G","H","I","J","K","L","M","N"]:
    ws.column_dimensions[column].width = 17

data = {
    "Product Name": ["ID", "Product 0", "Product 1", "Product 2", "Product 3", 
                    "Product 4","Product 5", "Product 6", "Product 7", 
                    "Product 8", "Product 9", "Product 10"],
    "Trazado": [30, 30, 30, 30, 20, 20, 20, 20, 20, 10, 10, 10],
    "Sales Month 2": [30, 30, 25, 25, 20, 19, 15, 15, 15, 10, 5, 5],
    "Sales Month 3": [30,30, 5, 10, 20, 10, 30, 30, 5, 5, 35, 10],
    "Sales Month 4": [30, 30, 5, 5, 35, 10, 30, 30, 5, 5, 35, 10],
    "Calificación": ['0', '0', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2'],
    "Sales Month 6": [ 20 , 30, 5, 5, 35, 10, 30, 30, 5, 5, 35, 10],
}

df = pd.DataFrame(data)
df = df.set_index(df.columns[0])

"""introduciendo los datos"""
for row in dataframe_to_rows(df, index = False, header=True):
    ws.append(row)

"""introducir columnas"""
columnas_1 = [i for i in string.ascii_uppercase] #letras de columnas
columnas_2 = [f'{i}{j}' for j in string.ascii_uppercase for i in string.ascii_uppercase] #letras de columnas
columnas = columnas_1 + columnas_2


columna = 4
ws.insert_cols(columna)
ws[f'{columnas[columna-1]}{ws.min_row}'].value = 'Total'


filas = numpy.arange(2,len(df)+1)
for row in ws.iter_rows(min_row=ws.min_row+1, max_row=ws.max_row, min_col=columna, max_col=columna):
    for cell in row:
        cell.value = f'= B{cell.row} + C{cell.row}'

########################
columna = 8
ws.insert_cols(columna)
ws[f'{columnas[columna-1]}{ws.min_row}'].value = 'Total'


filas = numpy.arange(2,len(df)+1)
for row in ws.iter_rows(min_row=ws.min_row+1, max_row=ws.max_row, min_col=columna, max_col=columna):
    for cell in row:
        cell.value = f'= D{cell.row} + E{cell.row} + F{cell.row} + G{cell.row}'

"""Estilos"""
#Fuente
fuente_letra_azul_oscuro = Font(bold= True, size='11',color= '00003366')

#Alineamiento
alinear_centro = Alignment(horizontal = 'center', vertical = 'center')

#borde
borde_4_lados = Border(
                        left=Side(style='thin'),  
                        right=Side(style='thin'), 
                        top=Side(style='thin'), 
                        bottom=Side(style='thin')
                        )
#relleno
relleno_azul_categoria = PatternFill(
                        start_color= '4787B7',
                        end_color= '4787B7',
                        fill_type= 'solid'
                        )
#encabezado                  
for row in ws.iter_rows(min_row=ws.min_row, max_row=1, min_col=ws.min_column, max_col=ws.max_column):
    for cell in row:
        cell.font = fuente_letra_azul_oscuro 
        cell.fill = relleno_azul_categoria      
        cell.alignment = alinear_centro
        cell.border = borde_4_lados

#cuerpo
for row in ws.iter_rows(min_row=ws.min_row+1, max_row=ws.max_row, min_col=ws.min_column, max_col=ws.max_column):
    for cell in row:
        # cell.font = fuente_letra_azul_oscuro 
        # cell.fill = relleno_azul_categoria      
        # cell.alignment = alinear_centro
        cell.border = borde_4_lados

#Trazado
# for row in ws.iter_rows(min_row=ws.min_row+1, max_row=ws.max_row, min_col=1, max_col=1):
#     for cell in row:
        # cell.value = f'Trazado {cell.value}'


In [20]:
wb.remove(wb["Sheet"])

In [66]:
"""merge cell of column B"""
table = []
for row in ws.iter_rows(min_row=ws.min_row, max_row=ws.max_row, min_col=ws.min_column, max_col=ws.max_column):
    for cell in row:
       table.append(cell)
table = numpy.array(table).reshape(ws.max_row, ws.max_column)

columna_B = [cell.value for cell in table[1:,1]]


conjunto = list(set(columna_B))[::-1]

number_rows = [columna_B.count(i) for i in conjunto]

cumsum_inverso = numpy.cumsum(number_rows[::-1])
cumsum = numpy.cumsum(number_rows)


for i,j in zip(cumsum_inverso, cumsum[::-1]):
    min_row = len(ws['B']) - i + 1
    max_row = j + 1
    ws.merge_cells(f'B{min_row}:B{max_row}')

In [18]:
"""head"""
fuente_backbone_index = Font(name='Calibri', size=20, bold=True,
font = Font(
        name='Calibri',
        size=11,
        bold=False,
        italic=False,
        vertAlign=None,
        underline='none',
        strike=False,
        color='FF000000')

alignment = Alignment(
                horizontal='general',
                vertical='bottom',
                text_rotation=0,
                wrap_text=False,
                shrink_to_fit=False,
                indent=0)
for cell in ws[1]:
    cell.

    
ws.row_dimensions[1].height = 30

IndentationError: unexpected indent (<ipython-input-18-6c79069ba8b3>, line 2)

In [21]:
wb.save('Prueba_14.xlsx')

In [25]:
"""separacion de trazados"""
table = []
for row in ws.iter_rows(min_row=ws.min_row, max_row=ws.max_row, min_col=ws.min_column, max_col=ws.max_column):
    for cell in row:
       table.append(cell)
table = numpy.array(table).reshape(ws.max_row, ws.max_column)

columna = [cell.value for cell in table[1:,0]]
columna

conjunto = list(set(columna))[::-1]

number_rows = [columna.count(i) for i in conjunto]
cumsum = numpy.cumsum(number_rows)

for i in cumsum[::-1]:
    row = i + 2
    ws.insert_rows(idx=row, amount=3)

In [67]:
"""merge cell with column trazados"""
table = []
for row in ws.iter_rows(min_row=ws.min_row, max_row=ws.max_row, min_col=ws.min_column, max_col=ws.max_column):
    for cell in row:
       table.append(cell)
table = numpy.array(table).reshape(ws.max_row, ws.max_column)

columna_A = [cell.value for cell in table[1:,0]]
conjunto = list(set(columna_A))[::-1]

number_rows = [columna_A.count(i) for i in conjunto]
space = number_rows[-1]/(len(number_rows[:-1]) - 1)


cumsum_inverso = numpy.cumsum(number_rows[:-1][::-1])
cumsum = numpy.cumsum(number_rows[:-1])
rows_used = len(ws['A']) + space

space_ = 0
min_row = []
for i in cumsum_inverso:
    space_ += space
    row = rows_used - i + 1 - space_
    min_row.append(row)

space__ = 0
max_row = []
for i in cumsum:
    row = i + 1 + space__
    space__ += space
    max_row.append(row)

for i, j in zip(min_row, max_row[::-1]):
    i, j = int(i), int(j)
    ws.merge_cells(f'A{i}:A{j}')


In [27]:
"""copiar filas a otras filas"""
# from copy import copy
# for cell in ws[1]:
#     for cell_ in ws[20]:
#         cell_.value = copy(cell.value)
        
ws.insert_cols(idx=1, amount=1)
ws.insert_rows(idx=1, amount=1)

In [28]:
# from copy import copy
# for cell,cell_ in zip(ws[1],ws[20]):
#     cell_.value = copy(cell.value)
#     cell_.style = copy(cell.style)
#     cell_.fill = copy(cell.fill)

In [68]:
wb.save('Prueba_12.xlsx')

In [30]:
"""separacion de trazados"""
# table = []
# for row in ws.iter_rows(min_row=ws.min_row, max_row=ws.max_row, min_col=ws.min_column, max_col=ws.max_column):
#     for cell in row:
#        table.append(cell)
# table = numpy.array(table).reshape(ws.max_row, ws.max_column)

# columna = [cell.value for cell in table[1:,0]]
# columna

# conjunto = list(set(columna))[::-1]

# number_rows = [columna.count(i) for i in conjunto]
# cumsum = numpy.cumsum(number_rows)

# for i in cumsum[::-1]:
#     row = i + 2
#     ws.insert_rows(idx=row, amount=3)



'separacion de trazados'